In [225]:
import pandas as pd

In [226]:
dataset = pd.read_csv("file:///home/rochet/corpora/GDNvsVD-data/GDNvsVD/GDN/LA_TRANSITION_ECOLOGIQUE.csv", delimiter=",")

/home/rochet/Logiciels/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
dataset.head(5)

,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,...,"QUXVlc3Rpb246MTUw - Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?",QUXVlc3Rpb246MTUx - Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?,"QUXVlc3Rpb246MTUy - Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?","QUXVlc3Rpb246MTUz - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à changer de mode de chauffage ?","QUXVlc3Rpb246MTU0 - Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?","QUXVlc3Rpb246MTU1 - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?","QUXVlc3Rpb246MjA3 - Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?",QUXVlc3Rpb246MTU3 - Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?,QUXVlc3Rpb246MTU4 - Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?,QUXVlc3Rpb246MTU5 - Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?
0,UHJvcG9zYWw6ZmZlN2MwNzktMWUyMC0xMWU5LTk0ZDItZm...,2-4,transition écologique,2019-01-22 09:37:49,2019-01-22 09:37:49,NaN,False,NaN,VXNlcjoxMTQwMTc0YS0xZTFmLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Enseignement du tri sélectif à l'école,Multiplier les centrales géothermiques
1,UHJvcG9zYWw6M2RmZmUxZjMtMWUyMS0xMWU5LTk0ZDItZm...,2-5,La surpopulation,2019-01-22 09:39:33,2019-01-22 09:39:33,NaN,False,NaN,VXNlcjpjOWYxZWQ1NS0xYzEwLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mettre en oeuvre au niveau national ses engage...,Les problèmes auxquels se trouve confronté l’e...
2,UHJvcG9zYWw6NDg1YWUxYjItMWUyMS0xMWU5LTk0ZDItZm...,2-6,climat,2019-01-22 09:39:50,2019-01-22 09:39:50,NaN,False,NaN,VXNlcjozZjlhNzAwOS0xYTc2LTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,"développer les transports en commun ,",NaN,Non,NaN,Oui,NaN,Les transports en commun|L'auto partage|Le tra...,NaN,NaN,NaN
3,UHJvcG9zYWw6NTk2Nzk5MmItMWUyMS0xMWU5LTk0ZDItZm...,2-7,POLLUTION AIR EAU,2019-01-22 09:40:19,2019-01-22 09:40:19,NaN,False,NaN,VXNlcjozOWQwNzJjNC0xZDEwLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,NaN,Plus de transports publics dans les petites co...,Non,NaN,Non,Le co-voiturage ne correspond pas toujours aux...,Les transports en commun,L'Etat,Elle n'a aucun pouvoir. Impossible de contrain...,NaN
4,UHJvcG9zYWw6ODU5NWMyOTYtMWUyMS0xMWU5LTk0ZDItZm...,2-8,Economie vs Ecologie,2019-01-22 09:41:33,2019-01-22 09:41:33,NaN,False,NaN,VXNlcjo3M2YxN2NlZS0xZDRiLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,"Aménagement de piste cyclable, développement d...","Détaxer le mode de chauffage écologique, une a...",Oui,une aide significative pour de l'éolien ou du ...,Non,Une piste cyclable pour éviter de risquer ma v...,Les transports en commun|Le covoiturage|Le vélo,"La commune, le département, la région",Demander à Nicolas Hulot,Une vrai politique écologique et non économique


In [227]:
questions = dataset.columns[11:]
dataset_filter = dataset[['title', 'authorType', *questions]]
for index, quest in enumerate(questions):
    print(index, quest.split("-", 1)[1])

0  Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?
1  Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?
2  Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?
3  Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?
4  À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?
5  Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?
6  Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?
7  Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?
8  Par rapport à votre mode de chauffage act

In [228]:
index_question = 3

quest, answers = questions[index_question], dataset_filter[quest].values

print("Question selectionnée :")
print(quest.split("-", 1)[1])

Question selectionnée :
 Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?


# App

In [229]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
nlp = spacy.load('fr')

In [230]:
def tokenizer(sent):
    return [tok.text for tok in nlp.tokenizer(sent)]

vectorizer = TfidfVectorizer(tokenizer=tokenizer)

answers = [an for an in answers if "nan" != str(an)]
answers_tfidf = vectorizer.fit_transform(answers)

In [ ]:
matrix = vectorizer.transform(answers)

In [253]:
def search(text):
    # vectorize text
    input_vector = vectorizer.transform([text])
    
    # compute cosinus similarity
    sims_result = cosine_similarity(input_vector, matrix)
    
    # zip in tuples (sentence, similarity)
    result = [*zip(answers, sims_result.squeeze())]
    
    # return most pertinent sentence
    pertinent = sorted(result, key=lambda x: x[1], reverse=True)
    
    
    result_list =  []
    for resp in pertinent:
        if len(resp[0]) < 100:
            result_list.append(resp)
        if len(result_list) == 5:
            break
    
    for an in result_list:
        print(an)
        print("=======")
        
        
    #pertinent = max(result, key=lambda x: x[1])[0]
    #return pertinent

In [ ]:
input_text = input("Question : ")
while input_text != "_stop":
    #print("Réponse :", search(input_text))
    search(input_text)
    input_text = input("Question : ")

Question : tu aimes ta vie ?
('As-tu regardé le ciel cette nuit ?', 0.23908433906027246)
('?', 0.1837577175766085)
('?', 0.1837577175766085)
('?', 0.1837577175766085)
('??', 0.1837577175766085)
Question : es tu concerné ?
('Non , je ne me sens plus concerné', 0.3172084855467649)
('développer encore plus es aides pour les changements de véhicules polluants.', 0.3165475879089786)
("Le Fret n'es pas assez développé en France", 0.31145656309034536)
('As-tu regardé le ciel cette nuit ?', 0.31111982467329674)
('Développer les aides incitatives, taxer  es poids lourds', 0.2990846618307971)
Question : Es-tu concerné par le changement climatisation?
('la climatisation', 0.41519939740636747)
('As-tu regardé le ciel cette nuit ?', 0.2924216062785875)
("Le Fret n'es pas assez développé en France", 0.26904302599435653)
('le transport aérien devrait le premier concerné par la transition écologique', 0.26782142996041897)
('Non , je ne me sens plus concerné', 0.2622087579547682)
Question : es tu conce